<a href="https://colab.research.google.com/github/konuctor/NASA_hackaton/blob/main/parserNASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Парсер данных для прогнозирования урожайности
Автоматически загружает:
1. NDVI данные с различных спутников
2. Статистику урожайности
3. Метеорологические данные
"""

import requests
import pandas as pd
import numpy as np
import json
import time
import os
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import zipfile
import io
import warnings
warnings.filterwarnings('ignore')

class CropYieldDataParser:
    def __init__(self):
        self.base_dir = "crop_yield_data"
        self.ensure_directories()

    def ensure_directories(self):
        """Создание необходимых директорий"""
        dirs = [
            self.base_dir,
            f"{self.base_dir}/ndvi",
            f"{self.base_dir}/statistics",
            f"{self.base_dir}/weather",
            f"{self.base_dir}/processed"
        ]
        for dir_path in dirs:
            os.makedirs(dir_path, exist_ok=True)

    def get_modis_ndvi_data(self, latitude: float, longitude: float,
                           start_date: str, end_date: str) -> pd.DataFrame:
        """
        Получение NDVI данных MODIS через NASA POWER API
        (альтернатива прямому доступу к MODIS)
        """
        print(f"Загрузка MODIS NDVI для координат ({latitude}, {longitude})")

        # NASA POWER API для получения вегетационных индексов
        base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"

        params = {
            'start': start_date.replace('-', ''),
            'end': end_date.replace('-', ''),
            'latitude': latitude,
            'longitude': longitude,
            'community': 'AG',  # Agricultural community
            'parameters': 'T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN',  # Температура, осадки, солнечная радиация
            'format': 'JSON',
            'header': 'true',
            'time-standard': 'UTC'
        }

        try:
            response = requests.get(base_url, params=params, timeout=30)
            response.raise_for_status()

            data = response.json()
            if 'properties' in data and 'parameter' in data['properties']:
                df_data = []
                parameters = data['properties']['parameter']

                # Получаем даты
                dates = list(parameters['T2M'].keys())

                for date in dates:
                    row = {
                        'date': pd.to_datetime(date),
                        'temperature': parameters.get('T2M', {}).get(date, np.nan),
                        'precipitation': parameters.get('PRECTOTCORR', {}).get(date, np.nan),
                        'solar_radiation': parameters.get('ALLSKY_SFC_SW_DWN', {}).get(date, np.nan)
                    }
                    df_data.append(row)

                df = pd.DataFrame(df_data)

                # Вычисляем простой вегетационный индекс на основе температуры и осадков
                df['ndvi_estimated'] = self.calculate_estimated_ndvi(df)

                return df
            else:
                print("Ошибка в структуре данных NASA POWER")
                return pd.DataFrame()

        except Exception as e:
            print(f"Ошибка при получении данных NASA POWER: {e}")
            return pd.DataFrame()

    def calculate_estimated_ndvi(self, weather_df: pd.DataFrame) -> pd.Series:
        """
        Расчет оценочного NDVI на основе метеорологических данных
        Простая эмпирическая формула
        """
        # Нормализация параметров
        temp_norm = (weather_df['temperature'] - weather_df['temperature'].min()) / \
                   (weather_df['temperature'].max() - weather_df['temperature'].min())

        precip_norm = weather_df['precipitation'] / weather_df['precipitation'].max()
        solar_norm = weather_df['solar_radiation'] / weather_df['solar_radiation'].max()

        # Простая формула для оценки NDVI (0-1)
        ndvi_est = 0.4 * temp_norm + 0.4 * precip_norm + 0.2 * solar_norm
        ndvi_est = np.clip(ndvi_est, 0, 1)  # Ограничиваем значения 0-1

        return ndvi_est

    def get_sentinel_ndvi_data(self, bbox: Tuple[float, float, float, float],
                              start_date: str, end_date: str) -> pd.DataFrame:
        """
        Получение данных Sentinel-2 через Copernicus API (требует регистрации)
        Здесь используем публичные данные для демонстрации
        """
        print(f"Загрузка Sentinel-2 NDVI для области {bbox}")

        # Для демонстрации генерируем синтетические данные Sentinel
        dates = pd.date_range(start=start_date, end=end_date, freq='10D')  # Каждые 10 дней

        synthetic_data = []
        for date in dates:
            # Генерируем реалистичные значения NDVI с сезонностью
            day_of_year = date.timetuple().tm_yday
            seasonal_factor = 0.3 + 0.4 * np.sin(2 * np.pi * (day_of_year - 80) / 365)  # Пик летом
            noise = np.random.normal(0, 0.05)  # Небольшой шум
            ndvi_value = np.clip(seasonal_factor + noise, 0, 1)

            synthetic_data.append({
                'date': date,
                'ndvi_sentinel': ndvi_value,
                'cloud_cover': np.random.uniform(0, 30),  # Облачность в %
                'bbox': bbox
            })

        return pd.DataFrame(synthetic_data)

    def get_russian_crop_statistics(self, year_start: int = 2010, year_end: int = 2023) -> pd.DataFrame:
        """
        Парсинг статистики урожайности с открытых источников
        """
        print(f"Загрузка статистики урожайности РФ за {year_start}-{year_end}")

        # Список регионов из статьи
        regions = [
            'Московская', 'Воронежская', 'Владимирская', 'Ивановская',
            'Нижегородская', 'Чувашская', 'Мордовия', 'Рязанская',
            'Тульская', 'Орловская', 'Курская', 'Тамбовская',
            'Липецкая', 'Пензенская'
        ]

        crops = ['Пшеница', 'Картофель', 'Овощи']

        # Генерируем реалистичные данные на основе статистики из статьи
        statistics_data = []

        for year in range(year_start, year_end + 1):
            for region in regions:
                for crop in crops:
                    # Базовые значения урожайности из статьи (центнеры с га)
                    base_yields = {
                        'Пшеница': np.random.normal(25, 5),     # 15-40 ц/га
                        'Картофель': np.random.normal(120, 20),  # 80-200 ц/га
                        'Овощи': np.random.normal(180, 30)      # 100-300 ц/га
                    }

                    # Добавляем тренд роста (как в статье - 7.4% за 10 лет)
                    trend_factor = 1 + (year - year_start) * 0.007

                    # Региональные коэффициенты (из статьи)
                    regional_factors = {
                        'Липецкая': 1.02,      # Самая плодородная
                        'Московская': 1.0,     # Базовая
                        'Пензенская': 0.59,    # Наименее плодородная
                    }

                    regional_factor = regional_factors.get(region, np.random.uniform(0.7, 1.1))

                    # Случайные климатические вариации
                    climate_factor = np.random.normal(1.0, 0.15)

                    # Аномальные годы (засуха 2010)
                    if year == 2010:
                        climate_factor *= 0.6

                    yield_value = base_yields[crop] * trend_factor * regional_factor * climate_factor
                    yield_value = max(yield_value, 0)  # Не может быть отрицательной

                    statistics_data.append({
                        'year': year,
                        'region': region,
                        'crop': crop,
                        'yield_centner_per_ha': round(yield_value, 1),
                        'trend_factor': trend_factor,
                        'regional_factor': regional_factor,
                        'climate_factor': climate_factor
                    })

        return pd.DataFrame(statistics_data)

    def get_weather_data(self, latitude: float, longitude: float,
                        start_date: str, end_date: str) -> pd.DataFrame:
        """
        Получение метеорологических данных через Open-Meteo API (бесплатно)
        """
        print(f"Загрузка погодных данных для ({latitude}, {longitude})")

        url = "https://archive-api.open-meteo.com/v1/era5"

        params = {
            'latitude': latitude,
            'longitude': longitude,
            'start_date': start_date,
            'end_date': end_date,
            'daily': 'temperature_2m_mean,precipitation_sum,shortwave_radiation_sum',
            'timezone': 'Europe/Moscow'
        }

        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()

            data = response.json()

            if 'daily' in data:
                df = pd.DataFrame({
                    'date': pd.to_datetime(data['daily']['time']),
                    'temperature_mean': data['daily']['temperature_2m_mean'],
                    'precipitation': data['daily']['precipitation_sum'],
                    'solar_radiation': data['daily']['shortwave_radiation_sum']
                })

                return df
            else:
                print("Нет данных в ответе Open-Meteo")
                return pd.DataFrame()

        except Exception as e:
            print(f"Ошибка при получении данных Open-Meteo: {e}")
            # Возвращаем синтетические данные в случае ошибки
            return self.generate_synthetic_weather(start_date, end_date)

    def generate_synthetic_weather(self, start_date: str, end_date: str) -> pd.DataFrame:
        """Генерация синтетических погодных данных"""
        dates = pd.date_range(start=start_date, end=end_date, freq='D')

        weather_data = []
        for date in dates:
            day_of_year = date.timetuple().tm_yday

            # Температура с сезонностью
            temp_base = 5 + 15 * np.sin(2 * np.pi * (day_of_year - 80) / 365)
            temperature = temp_base + np.random.normal(0, 5)

            # Осадки (больше весной и осенью)
            precip_base = 2 + 3 * np.sin(2 * np.pi * (day_of_year - 120) / 365)**2
            precipitation = max(0, precip_base + np.random.exponential(1))

            # Солнечная радиация
            solar_base = 100 + 150 * np.sin(2 * np.pi * (day_of_year - 80) / 365)
            solar_radiation = max(0, solar_base + np.random.normal(0, 20))

            weather_data.append({
                'date': date,
                'temperature_mean': round(temperature, 1),
                'precipitation': round(precipitation, 1),
                'solar_radiation': round(solar_radiation, 1)
            })

        return pd.DataFrame(weather_data)

    def download_all_data(self, regions_coords: Dict[str, Tuple[float, float]],
                         years: List[int]) -> Dict[str, pd.DataFrame]:
        """
        Загрузка всех данных для указанных регионов и лет
        """
        all_data = {
            'crop_statistics': pd.DataFrame(),
            'weather_data': pd.DataFrame(),
            'ndvi_data': pd.DataFrame(),
            'sentinel_data': pd.DataFrame()
        }

        print("=== Начинаем загрузку данных ===")

        # 1. Статистика урожайности
        print("\n1. Загрузка статистики урожайности...")
        crop_stats = self.get_russian_crop_statistics(min(years), max(years))
        all_data['crop_statistics'] = crop_stats
        crop_stats.to_csv(f"{self.base_dir}/statistics/crop_statistics.csv", index=False)
        print(f"Сохранено {len(crop_stats)} записей статистики")

        # 2. Погодные данные и NDVI для каждого региона
        weather_combined = []
        ndvi_combined = []
        sentinel_combined = []

        for region, (lat, lon) in regions_coords.items():
            print(f"\n2. Обработка региона: {region}")

            for year in years:
                start_date = f"{year}-03-01"
                end_date = f"{year}-07-31"

                # Погодные данные
                weather_df = self.get_weather_data(lat, lon, start_date, end_date)
                if not weather_df.empty:
                    weather_df['region'] = region
                    weather_df['year'] = year
                    weather_combined.append(weather_df)

                # NDVI данные (MODIS эквivalent)
                ndvi_df = self.get_modis_ndvi_data(lat, lon, start_date, end_date)
                if not ndvi_df.empty:
                    ndvi_df['region'] = region
                    ndvi_df['year'] = year
                    ndvi_combined.append(ndvi_df)

                # Sentinel данные (более детальные)
                bbox = (lon-0.1, lat-0.1, lon+0.1, lat+0.1)  # Небольшая область вокруг точки
                sentinel_df = self.get_sentinel_ndvi_data(bbox, start_date, end_date)
                if not sentinel_df.empty:
                    sentinel_df['region'] = region
                    sentinel_df['year'] = year
                    sentinel_combined.append(sentinel_df)

                time.sleep(0.5)  # Пауза между запросами

        # Объединяем данные
        if weather_combined:
            all_data['weather_data'] = pd.concat(weather_combined, ignore_index=True)
            all_data['weather_data'].to_csv(f"{self.base_dir}/weather/weather_data.csv", index=False)
            print(f"Сохранено {len(all_data['weather_data'])} записей погоды")

        if ndvi_combined:
            all_data['ndvi_data'] = pd.concat(ndvi_combined, ignore_index=True)
            all_data['ndvi_data'].to_csv(f"{self.base_dir}/ndvi/modis_ndvi.csv", index=False)
            print(f"Сохранено {len(all_data['ndvi_data'])} записей NDVI")

        if sentinel_combined:
            all_data['sentinel_data'] = pd.concat(sentinel_combined, ignore_index=True)
            all_data['sentinel_data'].to_csv(f"{self.base_dir}/ndvi/sentinel_ndvi.csv", index=False)
            print(f"Сохранено {len(all_data['sentinel_data'])} записей Sentinel")

        print("\n=== Загрузка завершена ===")
        return all_data

    def create_analysis_dataset(self, all_data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """
        Создание объединенного датасета для анализа
        """
        print("\nСоздание объединенного датасета...")

        crop_stats = all_data['crop_statistics']
        weather_data = all_data['weather_data']
        ndvi_data = all_data['ndvi_data']

        # Агрегируем погодные данные по месяцам
        if not weather_data.empty:
            weather_data['month'] = weather_data['date'].dt.month
            weather_monthly = weather_data.groupby(['region', 'year', 'month']).agg({
                'temperature_mean': 'mean',
                'precipitation': 'sum',
                'solar_radiation': 'mean'
            }).reset_index()

        # Агрегируем NDVI по месяцам
        if not ndvi_data.empty:
            ndvi_data['month'] = ndvi_data['date'].dt.month
            ndvi_monthly = ndvi_data.groupby(['region', 'year', 'month']).agg({
                'ndvi_estimated': 'mean'
            }).reset_index()

        # Объединяем все данные
        combined_data = []

        for _, row in crop_stats.iterrows():
            region = row['region']
            year = row['year']
            crop = row['crop']
            yield_val = row['yield_centner_per_ha']

            record = {
                'region': region,
                'year': year,
                'crop': crop,
                'yield_centner_per_ha': yield_val
            }

            # Добавляем погодные данные по месяцам (март-июль)
            for month in range(3, 8):  # Март-июль
                if not weather_data.empty:
                    weather_month = weather_monthly[
                        (weather_monthly['region'] == region) &
                        (weather_monthly['year'] == year) &
                        (weather_monthly['month'] == month)
                    ]

                    if not weather_month.empty:
                        record[f'temp_month_{month}'] = weather_month['temperature_mean'].iloc[0]
                        record[f'precip_month_{month}'] = weather_month['precipitation'].iloc[0]
                        record[f'solar_month_{month}'] = weather_month['solar_radiation'].iloc[0]

                # Добавляем NDVI данные по месяцам
                if not ndvi_data.empty:
                    ndvi_month = ndvi_monthly[
                        (ndvi_monthly['region'] == region) &
                        (ndvi_monthly['year'] == year) &
                        (ndvi_monthly['month'] == month)
                    ]

                    if not ndvi_month.empty:
                        record[f'ndvi_month_{month}'] = ndvi_month['ndvi_estimated'].iloc[0]

            combined_data.append(record)

        final_dataset = pd.DataFrame(combined_data)
        final_dataset.to_csv(f"{self.base_dir}/processed/combined_dataset.csv", index=False)

        print(f"Создан итоговый датасет: {final_dataset.shape}")
        print(f"Столбцы: {list(final_dataset.columns)}")

        return final_dataset


def main():
    """Основная функция для демонстрации работы парсера"""

    # Инициализация парсера
    parser = CropYieldDataParser()

    # Координаты центров регионов (примерные)
    regions_coords = {
        'Московская': (55.7558, 37.6176),
        'Воронежская': (51.6720, 39.1843),
        'Владимирская': (56.1366, 40.3966),
        'Ивановская': (57.0000, 41.0000),
        'Нижегородская': (56.3287, 44.0020),
        'Рязанская': (54.6269, 39.6916),
        'Тульская': (54.1961, 37.6182),
        'Орловская': (52.9651, 36.0785),
        'Курская': (51.7373, 36.1873),
        'Тамбовская': (52.7210, 41.4520),
        'Липецкая': (52.6031, 39.5708),
        'Пензенская': (53.2001, 45.0000)
    }

    # Года для анализа
    years = [2018, 2019, 2020, 2021, 2022, 2023]

    print("Запуск парсера данных для прогнозирования урожайности")
    print(f"Регионы: {len(regions_coords)}")
    print(f"Года: {years}")

    # Загрузка всех данных
    all_data = parser.download_all_data(regions_coords, years)

    # Создание итогового датасета
    final_dataset = parser.create_analysis_dataset(all_data)

    # Вывод статистики
    print(f"\n=== РЕЗУЛЬТАТЫ ПАРСИНГА ===")
    print(f"Статистика урожайности: {len(all_data['crop_statistics'])} записей")
    print(f"Погодные данные: {len(all_data['weather_data'])} записей")
    print(f"NDVI данные: {len(all_data['ndvi_data'])} записей")
    print(f"Итоговый датасет: {final_dataset.shape}")
    print(f"\nВсе файлы сохранены в директории: {parser.base_dir}/")

    # Показать образец данных
    print(f"\n=== ОБРАЗЕЦ ИТОГОВОГО ДАТАСЕТА ===")
    print(final_dataset.head())

    return final_dataset, all_data


if __name__ == "__main__":
    dataset, data = main()

Запуск парсера данных для прогнозирования урожайности
Регионы: 12
Года: [2018, 2019, 2020, 2021, 2022, 2023]
=== Начинаем загрузку данных ===

1. Загрузка статистики урожайности...
Загрузка статистики урожайности РФ за 2018-2023
Сохранено 252 записей статистики

2. Обработка региона: Московская
Загрузка погодных данных для (55.7558, 37.6176)
Загрузка MODIS NDVI для координат (55.7558, 37.6176)
Загрузка Sentinel-2 NDVI для области (37.5176, 55.6558, 37.717600000000004, 55.8558)
Загрузка погодных данных для (55.7558, 37.6176)
Загрузка MODIS NDVI для координат (55.7558, 37.6176)
Загрузка Sentinel-2 NDVI для области (37.5176, 55.6558, 37.717600000000004, 55.8558)
Загрузка погодных данных для (55.7558, 37.6176)
Загрузка MODIS NDVI для координат (55.7558, 37.6176)
Загрузка Sentinel-2 NDVI для области (37.5176, 55.6558, 37.717600000000004, 55.8558)
Загрузка погодных данных для (55.7558, 37.6176)
Загрузка MODIS NDVI для координат (55.7558, 37.6176)
Загрузка Sentinel-2 NDVI для области (37.5176

In [ ]:
# -*- coding: utf-8 -*-
"""
# 🌾 ПРОГНОЗИРОВАНИЕ УРОЖАЙНОСТИ ПО СПУТНИКОВЫМ ДАННЫМ
## Реализация 4 моделей из научной статьи для Google Colab

Основано на статье: "Прогнозирование урожайности на основе многолетних космических наблюдений за динамикой развития вегетации"
"""

# ==========================================
# 📦 УСТАНОВКА И ИМПОРТ БИБЛИОТЕК
# ==========================================

# Устанавливаем необходимые библиотеки для Colab
!pip install plotly scikit-learn seaborn -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import json
import os
from datetime import datetime

# Настройка для красивых графиков
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Все библиотеки установлены и импортированы!")
print(f"📊 Версия pandas: {pd.__version__}")
print(f"🧮 Версия numpy: {np.__version__}")

# ==========================================
# 🎲 ГЕНЕРАЦИЯ ТЕСТОВЫХ ДАННЫХ (если нет парсера)
# ==========================================

def generate_synthetic_dataset(n_regions=14, n_years=10, n_crops=3):
    """
    Генерация синтетического датасета по образцу из статьи
    """
    print("🔄 Генерация синтетических данных...")

    # Регионы из статьи
    regions = [
        'Московская', 'Воронежская', 'Владимирская', 'Ивановская',
        'Нижегородская', 'Чувашская', 'Мордовия', 'Рязанская',
        'Тульская', 'Орловская', 'Курская', 'Тамбовская',
        'Липецкая', 'Пензенская'
    ]

    crops = ['Пшеница', 'Картофель', 'Овощи']
    years = range(2010, 2010 + n_years)

    data = []

    # Коэффициенты из статьи
    regional_coefficients = {
        'Липецкая': 1.02,      # Самая плодородная
        'Московская': 1.0,     # Базовая
        'Пензенская': 0.59,    # Наименее плодородная
    }

    # Базовые урожайности
    base_yields = {
        'Пшеница': 25,
        'Картофель': 120,
        'Овощи': 180
    }

    for region in regions:
        for crop in crops:
            for year in years:
                # Региональный коэффициент
                reg_coeff = regional_coefficients.get(region, np.random.uniform(0.7, 1.1))

                # Тренд роста (7.4% за 10 лет из статьи)
                trend = 1 + (year - 2010) * 0.0074

                # Климатические факторы
                climate = np.random.normal(1.0, 0.15)
                if year == 2010:  # Засушливый год
                    climate *= 0.6

                # Базовая урожайность
                base_yield = base_yields[crop]
                yield_value = base_yield * reg_coeff * trend * climate
                yield_value = max(yield_value, 0)

                # Генерируем NDVI данные по месяцам (март-июль)
                record = {
                    'region': region,
                    'year': year,
                    'crop': crop,
                    'yield_centner_per_ha': round(yield_value, 1)
                }

                # NDVI по месяцам с сезонностью
                for month in range(3, 8):  # Март-июль
                    # Сезонная компонента (пик в мае-июне)
                    seasonal_peak = 0.3 + 0.4 * np.sin(np.pi * (month - 3) / 4)

                    # Связь с урожайностью (корреляция)
                    yield_factor = (yield_value / base_yields[crop]) * 0.3

                    # Случайный шум
                    noise = np.random.normal(0, 0.05)

                    ndvi_value = np.clip(seasonal_peak + yield_factor + noise, 0.1, 0.9)
                    record[f'ndvi_month_{month}'] = round(ndvi_value, 3)

                    # Добавляем температуру и осадки
                    temp_base = 5 + 15 * np.sin(2 * np.pi * (month * 30 - 80) / 365)
                    record[f'temp_month_{month}'] = round(temp_base + np.random.normal(0, 3), 1)

                    precip = max(0, np.random.exponential(40))
                    record[f'precip_month_{month}'] = round(precip, 1)

                data.append(record)

    df = pd.DataFrame(data)
    print(f"✅ Создан датасет: {df.shape}")
    print(f"📋 Столбцы: {list(df.columns)}")

    return df

# ==========================================
# 📊 АНАЛИЗ ДАННЫХ
# ==========================================

def analyze_dataset(df):
    """Анализ загруженного датасета"""
    print("=" * 60)
    print("📊 АНАЛИЗ ДАТАСЕТА")
    print("=" * 60)

    print(f"🔢 Размер датасета: {df.shape}")
    print(f"📅 Года: {sorted(df['year'].unique())}")
    print(f"🌍 Регионы: {len(df['region'].unique())}")
    print(f"🌾 Культуры: {list(df['crop'].unique())}")

    # Статистика по урожайности
    print(f"\n📈 СТАТИСТИКА УРОЖАЙНОСТИ:")
    yield_stats = df.groupby('crop')['yield_centner_per_ha'].agg(['mean', 'std', 'min', 'max'])
    print(yield_stats.round(1))

    # Пропущенные значения
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n⚠️ Пропущенные значения:")
        print(missing[missing > 0])
    else:
        print("\n✅ Пропущенных значений нет")

    return df

def visualize_data_overview(df):
    """Обзорная визуализация данных"""

    # 1. Распределение урожайности по культурам
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Урожайность по культурам', 'Тренд по годам',
                       'Урожайность по регионам', 'NDVI динамика'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Бокс-плот по культурам
    for i, crop in enumerate(df['crop'].unique()):
        crop_data = df[df['crop'] == crop]['yield_centner_per_ha']
        fig.add_trace(
            go.Box(y=crop_data, name=crop, showlegend=False),
            row=1, col=1
        )

    # Тренд по годам
    yearly_trend = df.groupby(['year', 'crop'])['yield_centner_per_ha'].mean().reset_index()
    for crop in yearly_trend['crop'].unique():
        crop_trend = yearly_trend[yearly_trend['crop'] == crop]
        fig.add_trace(
            go.Scatter(x=crop_trend['year'], y=crop_trend['yield_centner_per_ha'],
                      name=crop, mode='lines+markers'),
            row=1, col=2
        )

    # Средняя урожайность по регионам
    regional_avg = df.groupby('region')['yield_centner_per_ha'].mean().sort_values(ascending=True)
    fig.add_trace(
        go.Bar(x=regional_avg.values, y=regional_avg.index,
               orientation='h', showlegend=False, marker_color='skyblue'),
        row=2, col=1
    )

    # NDVI динамика
    ndvi_cols = [col for col in df.columns if 'ndvi_month' in col]
    if ndvi_cols:
        months = [int(col.split('_')[-1]) for col in ndvi_cols]
        for crop in df['crop'].unique():
            crop_data = df[df['crop'] == crop]
            ndvi_values = [crop_data[col].mean() for col in ndvi_cols]
            fig.add_trace(
                go.Scatter(x=months, y=ndvi_values, name=f'NDVI {crop}',
                          mode='lines+markers'),
                row=2, col=2
            )

    fig.update_layout(height=800, title_text="📊 Обзор данных", showlegend=True)
    fig.show()

# ==========================================
# 🧠 МОДЕЛИ ПРОГНОЗИРОВАНИЯ
# ==========================================

class CropYieldModels:
    """Класс для реализации 4 моделей из статьи"""

    def __init__(self, df):
        self.df = df.copy()
        self.models = {}
        self.results = {}
        self.prepare_data()

    def prepare_data(self):
        """Подготовка данных для моделирования"""
        print("🔧 Подготовка данных для моделирования...")

        # Выделяем NDVI признаки
        self.ndvi_columns = [col for col in self.df.columns if 'ndvi_month' in col]

        # Создаем признаки для каждой модели
        self.X_base = self.df[self.ndvi_columns].fillna(self.df[self.ndvi_columns].mean())
        self.y = self.df['yield_centner_per_ha']

        # Дополнительные признаки
        self.df['region_encoded'] = pd.Categorical(self.df['region']).codes
        self.df['crop_encoded'] = pd.Categorical(self.df['crop']).codes
        self.df['year_trend'] = self.df['year'] - self.df['year'].min()

        print(f"✅ Подготовлено {len(self.ndvi_columns)} NDVI признаков")

    def model_1_global_linear(self):
        """
        Модель 1: Глобальная линейная модель
        Одинаковые коэффициенты для всех регионов и культур
        """
        print("\n🔵 Модель 1: Глобальная линейная модель")

        X = self.X_base
        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Кросс-валидация
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': self.ndvi_columns
        }

        self.models['model_1'] = model
        self.results['model_1'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def model_2_regional_linear(self):
        """
        Модель 2: Линейная модель для отдельных областей
        Разные коэффициенты для каждого региона
        """
        print("\n🟢 Модель 2: Региональная линейная модель")

        # Добавляем региональные дамми-переменные
        region_dummies = pd.get_dummies(self.df['region'], prefix='region')
        X = pd.concat([self.X_base, region_dummies], axis=1)
        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Кросс-валидация
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': list(X.columns)
        }

        self.models['model_2'] = model
        self.results['model_2'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def model_3_multiplicative(self):
        """
        Модель 3: Мультипликативная поправка для областей
        Базовая модель из статьи
        """
        print("\n🟡 Модель 3: Мультипликативная модель")

        # Создаем мультипликативные признаки
        X_mult = self.X_base.copy()

        # Добавляем взаимодействия NDVI с регионами
        for region in self.df['region'].unique():
            region_mask = (self.df['region'] == region).astype(int)
            for ndvi_col in self.ndvi_columns:
                X_mult[f'{ndvi_col}_x_{region}'] = self.X_base[ndvi_col] * region_mask

        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X_mult, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Кросс-валидация (с меньшим количеством фолдов из-за большого числа признаков)
        cv_scores = cross_val_score(model, X_mult, y, cv=3, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': list(X_mult.columns)
        }

        self.models['model_3'] = model
        self.results['model_3'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def model_4_trend_multiplicative(self):
        """
        Модель 4: Трендовая модель с мультипликативной поправкой
        Лучшая модель из статьи
        """
        print("\n🔴 Модель 4: Трендовая мультипликативная модель")

        # Создаем признаки с трендом
        X_trend = self.X_base.copy()

        # Добавляем временной тренд
        X_trend['year_trend'] = self.df['year_trend']
        X_trend['year_trend_squared'] = self.df['year_trend'] ** 2

        # Добавляем региональные коэффициенты
        region_dummies = pd.get_dummies(self.df['region'], prefix='region')
        X_trend = pd.concat([X_trend, region_dummies], axis=1)

        # Добавляем культурные коэффициенты
        crop_dummies = pd.get_dummies(self.df['crop'], prefix='crop')
        X_trend = pd.concat([X_trend, crop_dummies], axis=1)

        # Мультипликативные взаимодействия NDVI с трендом
        for ndvi_col in self.ndvi_columns:
            X_trend[f'{ndvi_col}_x_trend'] = X_trend[ndvi_col] * X_trend['year_trend']

        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X_trend, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Статистическая значимость (F-test как в статье)
        n = len(y_train)
        k = X_train.shape[1]
        adjusted_r2 = 1 - (1 - train_r2) * (n - 1) / (n - k - 1)
        f_statistic = (adjusted_r2 / (1 - adjusted_r2)) * ((n - k - 1) / k)

        # Кросс-валидация
        cv_scores = cross_val_score(model, X_trend, y, cv=3, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'adjusted_r2': adjusted_r2,
            'f_statistic': f_statistic,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': list(X_trend.columns),
            'n_params': k
        }

        self.models['model_4'] = model
        self.results['model_4'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Adjusted R²: {adjusted_r2:.3f}")
        print(f"   F-statistic: {f_statistic:.1f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def run_all_models(self):
        """Запуск всех 4 моделей"""
        print("🚀 ЗАПУСК ВСЕХ МОДЕЛЕЙ")
        print("=" * 60)

        self.model_1_global_linear()
        self.model_2_regional_linear()
        self.model_3_multiplicative()
        self.model_4_trend_multiplicative()

        return self.results

    def compare_models(self):
        """Сравнение всех моделей"""
        print("\n📊 СРАВНЕНИЕ МОДЕЛЕЙ")
        print("=" * 60)

        comparison_data = []
        for model_name, results in self.results.items():
            comparison_data.append({
                'Модель': model_name.replace('_', ' ').title(),
                'Train R²': results['train_r2'],
                'Test R²': results['test_r2'],
                'Test RMSE': results['test_rmse'],
                'CV R² Mean': results['cv_mean'],
                'CV R² Std': results['cv_std']
            })

        comparison_df = pd.DataFrame(comparison_data)
        print(comparison_df.round(3))

        return comparison_df

# ==========================================
# 📈 ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
# ==========================================

def visualize_model_comparison(results_dict):
    """Визуализация сравнения моделей"""

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Сравнение R²', 'Сравнение RMSE',
                       'Предсказания vs Реальные значения', 'Остатки'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    model_names = list(results_dict.keys())
    r2_scores = [results_dict[model]['test_r2'] for model in model_names]
    rmse_scores = [results_dict[model]['test_rmse'] for model in model_names]

    # 1. Сравнение R²
    fig.add_trace(
        go.Bar(x=[m.replace('_', ' ').title() for m in model_names],
               y=r2_scores, name='R²', marker_color='lightblue'),
        row=1, col=1
    )

    # 2. Сравнение RMSE
    fig.add_trace(
        go.Bar(x=[m.replace('_', ' ').title() for m in model_names],
               y=rmse_scores, name='RMSE', marker_color='lightcoral'),
        row=1, col=2
    )

    # 3. Предсказания vs реальные (лучшая модель)
    best_model = max(model_names, key=lambda x: results_dict[x]['test_r2'])
    best_results = results_dict[best_model]

    fig.add_trace(
        go.Scatter(x=best_results['y_test'], y=best_results['y_pred'],
                  mode='markers', name=f'Предсказания ({best_model})',
                  marker=dict(color='green', size=8)),
        row=2, col=1
    )

    # Линия идеального предсказания
    min_val = min(best_results['y_test'].min(), best_results['y_pred'].min())
    max_val = max(best_results['y_test'].max(), best_results['y_pred'].max())
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                  mode='lines', name='Идеальное предсказание',
                  line=dict(dash='dash', color='red')),
        row=2, col=1
    )

    # 4. Остатки
    residuals = best_results['y_test'] - best_results['y_pred']
    fig.add_trace(
        go.Scatter(x=best_results['y_pred'], y=residuals,
                  mode='markers', name='Остатки',
                  marker=dict(color='orange', size=8)),
        row=2, col=2
    )

    # Горизонтальная линия в 0
    fig.add_trace(
        go.Scatter(x=[best_results['y_pred'].min(), best_results['y_pred'].max()],
                  y=[0, 0], mode='lines', name='Нулевая линия',
                  line=dict(dash='dash', color='black')),
        row=2, col=2
    )

    fig.update_layout(
        height=800,
        title_text="🏆 Сравнение моделей прогнозирования урожайности",
        showlegend=True
    )

    # Обновляем подписи осей
    fig.update_xaxes(title_text="Модели", row=1, col=1)
    fig.update_yaxes(title_text="R²", row=1, col=1)
    fig.update_xaxes(title_text="Модели", row=1, col=2)
    fig.update_yaxes(title_text="RMSE", row=1, col=2)
    fig.update_xaxes(title_text="Реальная урожайность", row=2, col=1)
    fig.update_yaxes(title_text="Предсказанная урожай")

✅ Все библиотеки установлены и импортированы!
📊 Версия pandas: 2.2.2
🧮 Версия numpy: 2.0.2


In [ ]:
# -*- coding: utf-8 -*-
"""
# 🌾 ПРОГНОЗИРОВАНИЕ УРОЖАЙНОСТИ ПО СПУТНИКОВЫМ ДАННЫМ
## Реализация 4 моделей из научной статьи для Google Colab

Основано на статье: "Прогнозирование урожайности на основе многолетних космических наблюдений за динамикой развития вегетации"
"""

# ==========================================
# 📦 УСТАНОВКА И ИМПОРТ БИБЛИОТЕК
# ==========================================

# Устанавливаем необходимые библиотеки для Colab
!pip install plotly scikit-learn seaborn -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import json
import os
from datetime import datetime

# Настройка для красивых графиков
plt.style.use('default')  # Изменено с 'seaborn-v0_8' на 'default'
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Все библиотеки установлены и импортированы!")
print(f"📊 Версия pandas: {pd.__version__}")
print(f"🧮 Версия numpy: {np.__version__}")

# ==========================================
# 🎲 ГЕНЕРАЦИЯ ТЕСТОВЫХ ДАННЫХ (если нет парсера)
# ==========================================

def generate_synthetic_dataset(n_regions=14, n_years=10, n_crops=3):
    """
    Генерация синтетического датасета по образцу из статьи
    """
    print("🔄 Генерация синтетических данных...")

    # Регионы из статьи
    regions = [
        'Московская', 'Воронежская', 'Владимирская', 'Ивановская',
        'Нижегородская', 'Чувашская', 'Мордовия', 'Рязанская',
        'Тульская', 'Орловская', 'Курская', 'Тамбовская',
        'Липецкая', 'Пензенская'
    ]

    crops = ['Пшеница', 'Картофель', 'Овощи']
    years = range(2010, 2010 + n_years)

    data = []

    # Коэффициенты из статьи
    regional_coefficients = {
        'Липецкая': 1.02,      # Самая плодородная
        'Московская': 1.0,     # Базовая
        'Пензенская': 0.59,    # Наименее плодородная
    }

    # Базовые урожайности
    base_yields = {
        'Пшеница': 25,
        'Картофель': 120,
        'Овощи': 180
    }

    for region in regions:
        for crop in crops:
            for year in years:
                # Региональный коэффициент
                reg_coeff = regional_coefficients.get(region, np.random.uniform(0.7, 1.1))

                # Тренд роста (7.4% за 10 лет из статьи)
                trend = 1 + (year - 2010) * 0.0074

                # Климатические факторы
                climate = np.random.normal(1.0, 0.15)
                if year == 2010:  # Засушливый год
                    climate *= 0.6

                # Базовая урожайность
                base_yield = base_yields[crop]
                yield_value = base_yield * reg_coeff * trend * climate
                yield_value = max(yield_value, 0)

                # Генерируем NDVI данные по месяцам (март-июль)
                record = {
                    'region': region,
                    'year': year,
                    'crop': crop,
                    'yield_centner_per_ha': round(yield_value, 1)
                }

                # NDVI по месяцам с сезонностью
                for month in range(3, 8):  # Март-июль
                    # Сезонная компонента (пик в мае-июне)
                    seasonal_peak = 0.3 + 0.4 * np.sin(np.pi * (month - 3) / 4)

                    # Связь с урожайностью (корреляция)
                    yield_factor = (yield_value / base_yields[crop]) * 0.3

                    # Случайный шум
                    noise = np.random.normal(0, 0.05)

                    ndvi_value = np.clip(seasonal_peak + yield_factor + noise, 0.1, 0.9)
                    record[f'ndvi_month_{month}'] = round(ndvi_value, 3)

                    # Добавляем температуру и осадки
                    temp_base = 5 + 15 * np.sin(2 * np.pi * (month * 30 - 80) / 365)
                    record[f'temp_month_{month}'] = round(temp_base + np.random.normal(0, 3), 1)

                    precip = max(0, np.random.exponential(40))
                    record[f'precip_month_{month}'] = round(precip, 1)

                data.append(record)

    df = pd.DataFrame(data)
    print(f"✅ Создан датасет: {df.shape}")
    print(f"📋 Столбцы: {list(df.columns)}")

    return df

# ==========================================
# 📊 АНАЛИЗ ДАННЫХ
# ==========================================

def analyze_dataset(df):
    """Анализ загруженного датасета"""
    print("=" * 60)
    print("📊 АНАЛИЗ ДАТАСЕТА")
    print("=" * 60)

    print(f"🔢 Размер датасета: {df.shape}")
    print(f"📅 Года: {sorted(df['year'].unique())}")
    print(f"🌍 Регионы: {len(df['region'].unique())}")
    print(f"🌾 Культуры: {list(df['crop'].unique())}")

    # Статистика по урожайности
    print(f"\n📈 СТАТИСТИКА УРОЖАЙНОСТИ:")
    yield_stats = df.groupby('crop')['yield_centner_per_ha'].agg(['mean', 'std', 'min', 'max'])
    print(yield_stats.round(1))

    # Пропущенные значения
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n⚠️ Пропущенные значения:")
        print(missing[missing > 0])
    else:
        print("\n✅ Пропущенных значений нет")

    return df

def visualize_data_overview(df):
    """Обзорная визуализация данных"""

    # 1. Распределение урожайности по культурам
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Урожайность по культурам', 'Тренд по годам',
                       'Урожайность по регионам', 'NDVI динамика'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Бокс-плот по культурам
    for i, crop in enumerate(df['crop'].unique()):
        crop_data = df[df['crop'] == crop]['yield_centner_per_ha']
        fig.add_trace(
            go.Box(y=crop_data, name=crop, showlegend=False),
            row=1, col=1
        )

    # Тренд по годам
    yearly_trend = df.groupby(['year', 'crop'])['yield_centner_per_ha'].mean().reset_index()
    for crop in yearly_trend['crop'].unique():
        crop_trend = yearly_trend[yearly_trend['crop'] == crop]
        fig.add_trace(
            go.Scatter(x=crop_trend['year'], y=crop_trend['yield_centner_per_ha'],
                      name=crop, mode='lines+markers'),
            row=1, col=2
        )

    # Средняя урожайность по регионам
    regional_avg = df.groupby('region')['yield_centner_per_ha'].mean().sort_values(ascending=True)
    fig.add_trace(
        go.Bar(x=regional_avg.values, y=regional_avg.index,
               orientation='h', showlegend=False, marker_color='skyblue'),
        row=2, col=1
    )

    # NDVI динамика
    ndvi_cols = [col for col in df.columns if 'ndvi_month' in col]
    if ndvi_cols:
        months = [int(col.split('_')[-1]) for col in ndvi_cols]
        for crop in df['crop'].unique():
            crop_data = df[df['crop'] == crop]
            ndvi_values = [crop_data[col].mean() for col in ndvi_cols]
            fig.add_trace(
                go.Scatter(x=months, y=ndvi_values, name=f'NDVI {crop}',
                          mode='lines+markers'),
                row=2, col=2
            )

    fig.update_layout(height=800, title_text="📊 Обзор данных", showlegend=True)
    fig.show()

# ==========================================
# 🧠 МОДЕЛИ ПРОГНОЗИРОВАНИЯ
# ==========================================

class CropYieldModels:
    """Класс для реализации 4 моделей из статьи"""

    def __init__(self, df):
        self.df = df.copy()
        self.models = {}
        self.results = {}
        self.prepare_data()

    def prepare_data(self):
        """Подготовка данных для моделирования"""
        print("🔧 Подготовка данных для моделирования...")

        # Выделяем NDVI признаки
        self.ndvi_columns = [col for col in self.df.columns if 'ndvi_month' in col]

        # Создаем признаки для каждой модели
        self.X_base = self.df[self.ndvi_columns].fillna(self.df[self.ndvi_columns].mean())
        self.y = self.df['yield_centner_per_ha']

        # Дополнительные признаки
        self.df['region_encoded'] = pd.Categorical(self.df['region']).codes
        self.df['crop_encoded'] = pd.Categorical(self.df['crop']).codes
        self.df['year_trend'] = self.df['year'] - self.df['year'].min()

        print(f"✅ Подготовлено {len(self.ndvi_columns)} NDVI признаков")

    def model_1_global_linear(self):
        """
        Модель 1: Глобальная линейная модель
        Одинаковые коэффициенты для всех регионов и культур
        """
        print("\n🔵 Модель 1: Глобальная линейная модель")

        X = self.X_base
        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Кросс-валидация
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': self.ndvi_columns
        }

        self.models['model_1'] = model
        self.results['model_1'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def model_2_regional_linear(self):
        """
        Модель 2: Линейная модель для отдельных областей
        Разные коэффициенты для каждого региона
        """
        print("\n🟢 Модель 2: Региональная линейная модель")

        # Добавляем региональные дамми-переменные
        region_dummies = pd.get_dummies(self.df['region'], prefix='region')
        X = pd.concat([self.X_base, region_dummies], axis=1)
        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Кросс-валидация
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': list(X.columns)
        }

        self.models['model_2'] = model
        self.results['model_2'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def model_3_multiplicative(self):
        """
        Модель 3: Мультипликативная поправка для областей
        Базовая модель из статьи
        """
        print("\n🟡 Модель 3: Мультипликативная модель")

        # Создаем мультипликативные признаки
        X_mult = self.X_base.copy()

        # Добавляем взаимодействия NDVI с регионами
        for region in self.df['region'].unique():
            region_mask = (self.df['region'] == region).astype(int)
            for ndvi_col in self.ndvi_columns:
                X_mult[f'{ndvi_col}_x_{region}'] = self.X_base[ndvi_col] * region_mask

        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X_mult, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Кросс-валидация (с меньшим количеством фолдов из-за большого числа признаков)
        cv_scores = cross_val_score(model, X_mult, y, cv=3, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': list(X_mult.columns)
        }

        self.models['model_3'] = model
        self.results['model_3'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def model_4_trend_multiplicative(self):
        """
        Модель 4: Трендовая модель с мультипликативной поправкой
        Лучшая модель из статьи
        """
        print("\n🔴 Модель 4: Трендовая мультипликативная модель")

        # Создаем признаки с трендом
        X_trend = self.X_base.copy()

        # Добавляем временной тренд
        X_trend['year_trend'] = self.df['year_trend']
        X_trend['year_trend_squared'] = self.df['year_trend'] ** 2

        # Добавляем региональные коэффициенты
        region_dummies = pd.get_dummies(self.df['region'], prefix='region')
        X_trend = pd.concat([X_trend, region_dummies], axis=1)

        # Добавляем культурные коэффициенты
        crop_dummies = pd.get_dummies(self.df['crop'], prefix='crop')
        X_trend = pd.concat([X_trend, crop_dummies], axis=1)

        # Мультипликативные взаимодействия NDVI с трендом
        for ndvi_col in self.ndvi_columns:
            X_trend[f'{ndvi_col}_x_trend'] = X_trend[ndvi_col] * X_trend['year_trend']

        y = self.y

        # Разделение на обучение/тест
        X_train, X_test, y_train, y_test = train_test_split(
            X_trend, y, test_size=0.2, random_state=42, stratify=self.df['crop']
        )

        # Обучение модели
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказания
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        # Метрики
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        # Статистическая значимость (F-test как в статье)
        n = len(y_train)
        k = X_train.shape[1]
        adjusted_r2 = 1 - (1 - train_r2) * (n - 1) / (n - k - 1)
        f_statistic = (adjusted_r2 / (1 - adjusted_r2)) * ((n - k - 1) / k)

        # Кросс-валидация
        cv_scores = cross_val_score(model, X_trend, y, cv=3, scoring='r2')

        results = {
            'model': model,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'adjusted_r2': adjusted_r2,
            'f_statistic': f_statistic,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_test': y_test,
            'y_pred': y_pred_test,
            'feature_names': list(X_trend.columns),
            'n_params': k
        }

        self.models['model_4'] = model
        self.results['model_4'] = results

        print(f"   Train R²: {train_r2:.3f}")
        print(f"   Test R²:  {test_r2:.3f}")
        print(f"   Adjusted R²: {adjusted_r2:.3f}")
        print(f"   F-statistic: {f_statistic:.1f}")
        print(f"   Test RMSE: {test_rmse:.2f}")
        print(f"   CV R² (μ±σ): {cv_scores.mean():.3f}±{cv_scores.std():.3f}")

        return results

    def run_all_models(self):
        """Запуск всех 4 моделей"""
        print("🚀 ЗАПУСК ВСЕХ МОДЕЛЕЙ")
        print("=" * 60)

        self.model_1_global_linear()
        self.model_2_regional_linear()
        self.model_3_multiplicative()
        self.model_4_trend_multiplicative()

        return self.results

    def compare_models(self):
        """Сравнение всех моделей"""
        print("\n📊 СРАВНЕНИЕ МОДЕЛЕЙ")
        print("=" * 60)

        comparison_data = []
        for model_name, results in self.results.items():
            comparison_data.append({
                'Модель': model_name.replace('_', ' ').title(),
                'Train R²': results['train_r2'],
                'Test R²': results['test_r2'],
                'Test RMSE': results['test_rmse'],
                'CV R² Mean': results['cv_mean'],
                'CV R² Std': results['cv_std']
            })

        comparison_df = pd.DataFrame(comparison_data)
        print(comparison_df.round(3))

        return comparison_df

# ==========================================
# 📈 ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
# ==========================================

def visualize_model_comparison(results_dict):
    """Визуализация сравнения моделей"""

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Сравнение R²', 'Сравнение RMSE',
                       'Предсказания vs Реальные значения', 'Остатки'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    model_names = list(results_dict.keys())
    r2_scores = [results_dict[model]['test_r2'] for model in model_names]
    rmse_scores = [results_dict[model]['test_rmse'] for model in model_names]

    # 1. Сравнение R²
    fig.add_trace(
        go.Bar(x=[m.replace('_', ' ').title() for m in model_names],
               y=r2_scores, name='R²', marker_color='lightblue'),
        row=1, col=1
    )

    # 2. Сравнение RMSE
    fig.add_trace(
        go.Bar(x=[m.replace('_', ' ').title() for m in model_names],
               y=rmse_scores, name='RMSE', marker_color='lightcoral'),
        row=1, col=2
    )

    # 3. Предсказания vs реальные (лучшая модель)
    best_model = max(model_names, key=lambda x: results_dict[x]['test_r2'])
    best_results = results_dict[best_model]

    fig.add_trace(
        go.Scatter(x=best_results['y_test'], y=best_results['y_pred'],
                  mode='markers', name=f'Предсказания ({best_model})',
                  marker=dict(color='green', size=8)),
        row=2, col=1
    )

    # Линия идеального предсказания
    min_val = min(best_results['y_test'].min(), best_results['y_pred'].min())
    max_val = max(best_results['y_test'].max(), best_results['y_pred'].max())
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                  mode='lines', name='Идеальное предсказание',
                  line=dict(dash='dash', color='red')),
        row=2, col=1
    )

    # 4. Остатки
    residuals = best_results['y_test'] - best_results['y_pred']
    fig.add_trace(
        go.Scatter(x=best_results['y_pred'], y=residuals,
                  mode='markers', name='Остатки',
                  marker=dict(color='orange', size=8)),
        row=2, col=2
    )

    # Горизонтальная линия в 0
    fig.add_trace(
        go.Scatter(x=[best_results['y_pred'].min(), best_results['y_pred'].max()],
                  y=[0, 0], mode='lines', name='Нулевая линия',
                  line=dict(dash='dash', color='black')),
        row=2, col=2
    )

    fig.update_layout(
        height=800,
        title_text="🏆 Сравнение моделей прогнозирования урожайности",
        showlegend=True
    )

    # Обновляем подписи осей
    fig.update_xaxes(title_text="Модели", row=1, col=1)
    fig.update_yaxes(title_text="R²", row=1, col=1)
    fig.update_xaxes(title_text="Модели", row=1, col=2)
    fig.update_yaxes(title_text="RMSE", row=1, col=2)
    fig.update_xaxes(title_text="Реальная урожайность", row=2, col=1)
    fig.update_yaxes(title_text="Предсказанная урожайность", row=2, col=1)
    fig.update_xaxes(title_text="Предсказанная урожайность", row=2, col=2)
    fig.update_yaxes(title_text="Остатки", row=2, col=2)

    fig.show()

def visualize_feature_importance(results_dict):
    """Визуализация важности признаков"""

    # Берем лучшую модель
    best_model_name = max(results_dict.keys(), key=lambda x: results_dict[x]['test_r2'])
    best_model = results_dict[best_model_name]['model']
    feature_names = results_dict[best_model_name]['feature_names']

    # Коэффициенты модели как показатель важности
    coefficients = best_model.coef_

    # Берем топ-15 самых важных признаков по абсолютному значению
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': np.abs(coefficients),
        'coefficient': coefficients
    }).sort_values('importance', ascending=False).head(15)

    fig = go.Figure()

    colors = ['green' if coef > 0 else 'red' for coef in importance_df['coefficient']]

    fig.add_trace(go.Bar(
        x=importance_df['importance'],
        y=importance_df['feature'],
        orientation='h',
        marker_color=colors,
        text=[f'{coef:.3f}' for coef in importance_df['coefficient']],
        textposition='auto',
    ))

    fig.update_layout(
        title=f"🎯 Важность признаков - {best_model_name.replace('_', ' ').title()}",
        xaxis_title="Абсолютное значение коэффициента",
        yaxis_title="Признаки",
        height=600,
        margin=dict(l=200)
    )

    fig.show()

    return importance_df

def visualize_predictions_by_crop(df, results_dict):
    """Визуализация предсказаний по культурам - ИСПРАВЛЕННАЯ ВЕРСИЯ"""

    best_model_name = max(results_dict.keys(), key=lambda x: results_dict[x]['test_r2'])
    best_results = results_dict[best_model_name]

    # Получаем индексы тестовых данных для привязки к культурам
    # (упрощенный подход - используем последние данные)
    test_indices = df.tail(len(best_results['y_test'])).index
    test_crops = df.loc[test_indices, 'crop'].values

    crops = df['crop'].unique()

    fig = make_subplots(
        rows=1, cols=len(crops),
        subplot_titles=[f'Предсказания - {crop}' for crop in crops]
    )

    for i, crop in enumerate(crops, 1):
        crop_mask = test_crops == crop
        if crop_mask.sum() > 0:
            crop_real = best_results['y_test'][crop_mask]
            crop_pred = best_results['y_pred'][crop_mask]

            # Scatter plot
            fig.add_trace(
                go.Scatter(x=crop_real, y=crop_pred, mode='markers',
                          name=f'{crop}', marker=dict(size=8)),
                row=1, col=i
            )

            # Идеальная линия
            if len(crop_real) > 0:
                min_val = min(crop_real.min(), crop_pred.min())
                max_val = max(crop_real.max(), crop_pred.max())
                fig.add_trace(
                    go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                              mode='lines', line=dict(dash='dash', color='red'),
                              name='Идеальное', showlegend=(i==1)),
                    row=1, col=i
                )

                # R² для данной культуры
                if len(crop_real) > 1:  # Нужно минимум 2 точки для R²
                    crop_r2 = r2_score(crop_real, crop_pred)
                    # Исправленный способ добавления аннотации
                    fig.add_annotation(
                        x=0.05, y=0.95,
                        text=f'R² = {crop_r2:.3f}',
                        showarrow=False,
                        xref=f'x{i if i > 1 else ""} domain',
                        yref=f'y{i if i > 1 else ""} domain',
                        bgcolor='white', bordercolor='black'
                    )

    fig.update_layout(
        height=400,
        title_text=f"📈 Предсказания по культурам - {best_model_name.replace('_', ' ').title()}",
        showlegend=True
    )

    for i in range(1, len(crops) + 1):
        fig.update_xaxes(title_text="Реальная урожайность", row=1, col=i)
        fig.update_yaxes(title_text="Предсказанная урожайность", row=1, col=i)

    fig.show()

def create_statistical_report(results_dict):
    """Создание статистического отчета как в статье"""

    print("\n📋 СТАТИСТИЧЕСКИЙ ОТЧЕТ")
    print("=" * 70)

    for model_name, results in results_dict.items():
        print(f"\n🔸 {model_name.replace('_', ' ').title()}")
        print("-" * 40)

        # Основные метрики
        print(f"R² (нескорректированный): {results['train_r2']:.3f}")
        print(f"R² (тестовый): {results['test_r2']:.3f}")
        print(f"RMSE: {results['test_rmse']:.2f} ц/га")
        print(f"Кросс-валидация R²: {results['cv_mean']:.3f} ± {results['cv_std']:.3f}")

        # Дополнительные метрики для модели 4 (как в статье)
        if 'adjusted_r2' in results:
            print(f"R² (скорректированный): {results['adjusted_r2']:.3f}")
            print(f"F-статистика: {results['f_statistic']:.1f}")

            # Проверка значимости (α = 0.05, табличное значение ≈ 1.67)
            is_significant = results['f_statistic'] > 1.67
            print(f"Статистическая значимость (α=0.05): {'✅ Да' if is_significant else '❌ Нет'}")

        print(f"Количество признаков: {len(results['feature_names'])}")

    # Итоговая таблица как в статье
    print(f"\n📊 ИТОГОВАЯ ТАБЛИЦА СРАВНЕНИЯ")
    print("-" * 70)

    comparison_data = []
    for model_name, results in results_dict.items():
        row = {
            'Модель': model_name.replace('_', ' ').title(),
            'R² (тест)': f"{results['test_r2']:.3f}",
            'RMSE': f"{results['test_rmse']:.1f}",
            'CV R²': f"{results['cv_mean']:.3f}±{results['cv_std']:.3f}"
        }

        if 'adjusted_r2' in results:
            row['R² (скорр.)'] = f"{results['adjusted_r2']:.3f}"
            row['F-стат.'] = f"{results['f_statistic']:.1f}"

        comparison_data.append(row)

    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

    # Лучшая модель
    best_model = max(results_dict.keys(), key=lambda x: results_dict[x]['test_r2'])
    best_r2 = results_dict[best_model]['test_r2']
    best_rmse = results_dict[best_model]['test_rmse']

    print(f"\n🏆 ЛУЧШАЯ МОДЕЛЬ: {best_model.replace('_', ' ').title()}")
    print(f"   R²: {best_r2:.3f}")
    print(f"   RMSE: {best_rmse:.2f} ц/га")

    # Сравнение с результатами из статьи
    print(f"\n📄 СРАВНЕНИЕ С РЕЗУЛЬТАТАМИ СТАТЬИ:")
    print(f"   Статья - Модель 4: R² ≈ 0.58-0.85, RMSE ≈ 10%")
    print(f"   Наша модель: R² = {best_r2:.3f}, RMSE = {best_rmse:.1f} ц/га")

    return comparison_df

# ==========================================
# 🚀 ОСНОВНОЙ ЗАПУСК
# ==========================================

def main():
    """Основная функция для запуска всего пайплайна"""

    print("🌾" * 30)
    print("    ПРОГНОЗИРОВАНИЕ УРОЖАЙНОСТИ")
    print("       ПО СПУТНИКОВЫМ ДАННЫМ")
    print("🌾" * 30)

    # 1. Генерация или загрузка данных
    print("\n📊 ЭТАП 1: Подготовка данных")
    try:
        # Попытка загрузить данные из парсера
        df = pd.read_csv('/content/crop_yield_data/processed/combined_dataset.csv')
        print("✅ Данные загружены из парсера")
    except:
        # Генерация синтетических данных
        df = generate_synthetic_dataset()
        print("✅ Созданы синтетические данные")

    # 2. Анализ данных
    print("\n📊 ЭТАП 2: Анализ данных")
    df = analyze_dataset(df)

    # 3. Визуализация обзора данных
    print("\n📊 ЭТАП 3: Визуализация данных")
    visualize_data_overview(df)

    # 4. Создание и обучение моделей
    print("\n🧠 ЭТАП 4: Создание моделей")
    crop_models = CropYieldModels(df)
    results = crop_models.run_all_models()

    # 5. Сравнение моделей
    print("\n📊 ЭТАП 5: Сравнение моделей")
    comparison_df = crop_models.compare_models()

    # 6. Визуализация результатов
    print("\n📈 ЭТАП 6: Визуализация результатов")
    visualize_model_comparison(results)

    # 7. Анализ важности признаков
    print("\n🎯 ЭТАП 7: Важность признаков")
    importance_df = visualize_feature_importance(results)

    # 8. Предсказания по культурам
    print("\n📈 ЭТАП 8: Анализ по культурам")
    visualize_predictions_by_crop(df, results)

    # 9. Статистический отчет
    print("\n📋 ЭТАП 9: Статистический отчет")
    final_report = create_statistical_report(results)

    # 10. Экспорт результатов
    print("\n💾 ЭТАП 10: Сохранение результатов")

    # Сохранение в Google Drive (если подключен)
    try:
        from google.colab import drive
        drive.mount('/content/drive')

        # Сохраняем основные результаты
        comparison_df.to_csv('/content/drive/My Drive/crop_yield_results.csv', index=False)
        importance_df.to_csv('/content/drive/My Drive/feature_importance.csv', index=False)

        # Сохраняем предсказания лучшей модели
        best_model_name = max(results.keys(), key=lambda x: results[x]['test_r2'])
        best_results = results[best_model_name]

        predictions_df = pd.DataFrame({
            'real_yield': best_results['y_test'],
            'predicted_yield': best_results['y_pred'],
            'residuals': best_results['y_test'] - best_results['y_pred']
        })
        predictions_df.to_csv('/content/drive/My Drive/predictions.csv', index=False)

        print("✅ Результаты сохранены в Google Drive")

    except:
        print("⚠️ Google Drive не подключен, сохранение локально")
        comparison_df.to_csv('/content/crop_yield_results.csv', index=False)
        importance_df.to_csv('/content/feature_importance.csv', index=False)

    print("\n🎉 АНАЛИЗ ЗАВЕРШЕН!")
    print("=" * 50)

    return df, crop_models, results

# ==========================================
# 🎮 ИНТЕРАКТИВНЫЙ ИНТЕРФЕЙС
# ==========================================

def create_prediction_interface(crop_models, df):
    """Создание интерактивного интерфейса для прогнозирования"""

    print("\n🎮 ИНТЕРАКТИВНЫЙ ПРОГНОЗАТОР УРОЖАЙНОСТИ")
    print("=" * 50)

    def predict_yield(region, crop, ndvi_values):
        """Функция прогнозирования для новых данных"""

        # Используем лучшую модель
        best_model_name = max(crop_models.results.keys(),
                             key=lambda x: crop_models.results[x]['test_r2'])
        model = crop_models.models[best_model_name]

        # Подготавливаем входные данные (упрощенная версия)
        input_data = {}

        # NDVI значения
        for i, month in enumerate(range(3, 8)):  # март-июль
            if i < len(ndvi_values):
                input_data[f'ndvi_month_{month}'] = ndvi_values[i]
            else:
                input_data[f'ndvi_month_{month}'] = 0.5  # среднее значение

        # Преобразуем в DataFrame
        input_df = pd.DataFrame([input_data])

        # Заполняем недостающие признаки нулями (для совместимости с моделью)
        feature_names = crop_models.results[best_model_name]['feature_names']
        for feature in feature_names:
            if feature not in input_df.columns:
                if 'region' in feature and region.lower() in feature.lower():
                    input_df[feature] = 1
                elif 'crop' in feature and crop.lower() in feature.lower():
                    input_df[feature] = 1
                else:
                    input_df[feature] = 0

        # Приводим к правильному порядку столбцов
        input_df = input_df.reindex(columns=feature_names, fill_value=0)

        # Делаем прогноз
        prediction = model.predict(input_df)[0]

        return max(0, prediction)  # Урожайность не может быть отрицательной

    # Пример использования
    print("Пример прогнозирования:")
    print("-" * 30)

    # Тестовые данные
    test_region = "Московская"
    test_crop = "Пшеница"
    test_ndvi = [0.3, 0.5, 0.7, 0.6, 0.4]  # NDVI за март-июль

    predicted_yield = predict_yield(test_region, test_crop, test_ndvi)

    print(f"Регион: {test_region}")
    print(f"Культура: {test_crop}")
    print(f"NDVI (март-июль): {test_ndvi}")
    print(f"Прогнозируемая урожайность: {predicted_yield:.1f} ц/га")

    # Сравнение с историческими данными
    historical = df[(df['region'] == test_region) & (df['crop'] == test_crop)]
    if not historical.empty:
        avg_historical = historical['yield_centner_per_ha'].mean()
        print(f"Средняя историческая урожайность: {avg_historical:.1f} ц/га")
        print(f"Отклонение от исторической: {((predicted_yield/avg_historical - 1)*100):+.1f}%")

    return predict_yield

# ==========================================
# 📱 ЗАПУСК В COLAB
# ==========================================

if __name__ == "__main__":
    # Основной запуск
    df, crop_models, results = main()

    # Создание интерактивного интерфейса
    predict_function = create_prediction_interface(crop_models, df)

    print(f"\n📋 ИНСТРУКЦИИ ДЛЯ ДАЛЬНЕЙШЕГО ИСПОЛЬЗОВАНИЯ:")
    print("-" * 50)
    print("1. Используйте переменную 'df' для анализа данных")
    print("2. Используйте 'crop_models' для доступа к обученным моделям")
    print("3. Используйте 'results' для анализа метрик")
    print("4. Используйте predict_function() для новых прогнозов")
    print("\nПример нового прогноза:")
    print("predicted = predict_function('Воронежская', 'Картофель', [0.4, 0.6, 0.8, 0.7, 0.5])")

    print(f"\n🎯 ОСНОВНЫЕ РЕЗУЛЬТАТЫ:")
    print(f"Лучшая модель: {max(results.keys(), key=lambda x: results[x]['test_r2']).replace('_', ' ').title()}")
    print(f"Достигнутая точность: R² = {max(results[x]['test_r2'] for x in results.keys()):.3f}")
    print(f"Средняя ошибка: {min(results[x]['test_rmse'] for x in results.keys()):.1f} ц/га")

"""
# 🎉 ГОТОВО! КРАТКОЕ РУКОВОДСТВО:

## 📖 Как использовать этот код в Google Colab:

1. **Скопируйте весь код** в новую ячейку Colab
2. **Запустите ячейку** - код автоматически:
   - Установит библиотеки
   - Создаст синтетические данные (если нет реальных)
   - Обучит все 4 модели из статьи
   - Покажет интерактивные графики
   - Создаст отчет с метриками

3. **Изучите результаты** в интерактивных графиках
4. **Используйте функцию predict_function()** для новых прогнозов

## 🔧 Настройки:
- Измените параметры в generate_synthetic_dataset() для других данных
- Добавьте реальные данные заменив строку с pd.read_csv()
- Настройте параметры моделей в классе CropYieldModels

## 📊 Что получите:
- 4 модели прогнозирования (как в статье)
- Интерактивные графики сравнения
- Статистический отчет с F-тестами
- Анализ важности признаков
- Функцию для новых прогнозов

Код полностью готов для запуска в Google Colab! 🚀

## 🐛 ИСПРАВЛЕНИЯ:
- Убрана устаревшая тема seaborn-v0_8
- Исправлен синтаксис xref в аннотациях Plotly
- Добавлены дополнительные проверки для предотвращения ошибок
"""

✅ Все библиотеки установлены и импортированы!
📊 Версия pandas: 2.2.2
🧮 Версия numpy: 2.0.2
🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
    ПРОГНОЗИРОВАНИЕ УРОЖАЙНОСТИ
       ПО СПУТНИКОВЫМ ДАННЫМ
🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾

📊 ЭТАП 1: Подготовка данных
✅ Данные загружены из парсера

📊 ЭТАП 2: Анализ данных
📊 АНАЛИЗ ДАТАСЕТА
🔢 Размер датасета: (252, 24)
📅 Года: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
🌍 Регионы: 14
🌾 Культуры: ['Пшеница', 'Картофель', 'Овощи']

📈 СТАТИСТИКА УРОЖАЙНОСТИ:
            mean   std   min    max
crop                               
Картофель  102.4  26.2  56.9  170.9
Овощи      157.3  44.9  59.7  262.2
Пшеница     22.6   6.5   6.6   37.3

⚠️ Пропущенные значения:
temp_month_3      36
precip_month_3    36
solar_month_3     36
ndvi_month_3      36
temp_month_4      36
precip_month_4    36
solar_month_4     36
ndvi_month_4      36
temp_month_5      36
precip_month_5    36
solar_month_5     36
ndvi_month_5      36
temp_


🧠 ЭТАП 4: Создание моделей
🔧 Подготовка данных для моделирования...
✅ Подготовлено 5 NDVI признаков
🚀 ЗАПУСК ВСЕХ МОДЕЛЕЙ

🔵 Модель 1: Глобальная линейная модель
   Train R²: 0.005
   Test R²:  -0.026
   Test RMSE: 63.40
   CV R² (μ±σ): -0.012±0.013

🟢 Модель 2: Региональная линейная модель
   Train R²: 0.041
   Test R²:  -0.081
   Test RMSE: 65.05
   CV R² (μ±σ): 0.004±0.023

🟡 Модель 3: Мультипликативная модель
   Train R²: 0.166
   Test R²:  -1.236
   Test RMSE: 93.57
   CV R² (μ±σ): -0.591±0.333

🔴 Модель 4: Трендовая мультипликативная модель
   Train R²: 0.814
   Test R²:  0.802
   Adjusted R²: 0.782
   F-statistic: 21.2
   Test RMSE: 27.87
   CV R² (μ±σ): -0.026±0.813

📊 ЭТАП 5: Сравнение моделей

📊 СРАВНЕНИЕ МОДЕЛЕЙ
    Модель  Train R²  Test R²  Test RMSE  CV R² Mean  CV R² Std
0  Model 1     0.005   -0.026     63.404      -0.012      0.013
1  Model 2     0.041   -0.081     65.054       0.004      0.023
2  Model 3     0.166   -1.236     93.569      -0.591      0.333
3  Model 4


🎯 ЭТАП 7: Важность признаков



📈 ЭТАП 8: Анализ по культурам



📋 ЭТАП 9: Статистический отчет

📋 СТАТИСТИЧЕСКИЙ ОТЧЕТ

🔸 Model 1
----------------------------------------
R² (нескорректированный): 0.005
R² (тестовый): -0.026
RMSE: 63.40 ц/га
Кросс-валидация R²: -0.012 ± 0.013
Количество признаков: 5

🔸 Model 2
----------------------------------------
R² (нескорректированный): 0.041
R² (тестовый): -0.081
RMSE: 65.05 ц/га
Кросс-валидация R²: 0.004 ± 0.023
Количество признаков: 19

🔸 Model 3
----------------------------------------
R² (нескорректированный): 0.166
R² (тестовый): -1.236
RMSE: 93.57 ц/га
Кросс-валидация R²: -0.591 ± 0.333
Количество признаков: 75

🔸 Model 4
----------------------------------------
R² (нескорректированный): 0.814
R² (тестовый): 0.802
RMSE: 27.87 ц/га
Кросс-валидация R²: -0.026 ± 0.813
R² (скорректированный): 0.782
F-статистика: 21.2
Статистическая значимость (α=0.05): ✅ Да
Количество признаков: 29

📊 ИТОГОВАЯ ТАБЛИЦА СРАВНЕНИЯ
----------------------------------------------------------------------
 Модель R² (тест) RMSE  

'\n# 🎉 ГОТОВО! КРАТКОЕ РУКОВОДСТВО:\n\n## 📖 Как использовать этот код в Google Colab:\n\n1. **Скопируйте весь код** в новую ячейку Colab\n2. **Запустите ячейку** - код автоматически:\n   - Установит библиотеки\n   - Создаст синтетические данные (если нет реальных)\n   - Обучит все 4 модели из статьи\n   - Покажет интерактивные графики\n   - Создаст отчет с метриками\n\n3. **Изучите результаты** в интерактивных графиках\n4. **Используйте функцию predict_function()** для новых прогнозов\n\n## 🔧 Настройки:\n- Измените параметры в generate_synthetic_dataset() для других данных\n- Добавьте реальные данные заменив строку с pd.read_csv()\n- Настройте параметры моделей в классе CropYieldModels\n\n## 📊 Что получите:\n- 4 модели прогнозирования (как в статье)\n- Интерактивные графики сравнения\n- Статистический отчет с F-тестами\n- Анализ важности признаков\n- Функцию для новых прогнозов\n\nКод полностью готов для запуска в Google Colab! 🚀\n\n## 🐛 ИСПРАВЛЕНИЯ:\n- Убрана устаревшая тема seaborn-v

In [ ]:
crop = '/content/crop_yield_data/ndvi/modis_ndvi.csv'
crop = pd.read_csv(crop)

In [ ]:
crop.head()

,date,temperature,precipitation,solar_radiation,ndvi_estimated,region,year
0,2018-03-01,-18.30,0.02,10.17,0.068553,Московская,2018
1,2018-03-02,-14.70,0.28,7.99,0.093586,Московская,2018
2,2018-03-03,-12.75,1.24,9.92,0.142072,Московская,2018
3,2018-03-04,-12.20,10.77,5.42,0.280873,Московская,2018
4,2018-03-05,-12.11,0.84,10.12,0.142803,Московская,2018


In [ ]:
crop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016 entries, 0 to 11015
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             11016 non-null  object 
 1   temperature      11016 non-null  float64
 2   precipitation    11016 non-null  float64
 3   solar_radiation  11016 non-null  float64
 4   ndvi_estimated   11016 non-null  float64
 5   region           11016 non-null  object 
 6   year             11016 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 602.6+ KB


In [ ]:
crop.region.value_counts()

,count
region,
Московская,918
Воронежская,918
Владимирская,918
Ивановская,918
Нижегородская,918
Рязанская,918
Тульская,918
Орловская,918
Курская,918


In [ ]:
crop.solar_radiation.value_counts()

,count
solar_radiation,
16.84,15
14.64,15
22.46,14
14.83,14
23.09,13
...,...
26.06,1
26.73,1
30.29,1


In [ ]:
crop.solar_radiation.describe()

,solar_radiation
count,11016.000000
mean,17.480365
std,6.609556
min,2.330000
25%,12.470000
50%,17.390000
75%,22.552500
max,31.240000
